In [12]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import LineString

In [ ]:
df = gpd.read_parquet('ambulance_nonmed2med_wo_break.parquet')
df.head(5)

In [14]:
from datetime import datetime
from shapely.geometry import Point

def get_segment_speeds(line, time_list):
    """
    Returns a list of tuples: (segment_start_point, speed_mps)
    """
    speeds = []
    coords = list(line.coords)
    for i in range(len(coords) - 1):
        pt1 = Point(coords[i])
        pt2 = Point(coords[i+1])
        try:
            t1 = datetime.fromisoformat(time_list[i])
            t2 = datetime.fromisoformat(time_list[i+1])
            dt = (t2 - t1).total_seconds()
            if dt > 0:
                dist = pt1.distance(pt2)
                speed = dist / dt
                speeds.append((pt1, speed))
        except Exception:
            continue
    return speeds

In [15]:
if 'SegmentSpeeds' not in df.columns:
    df['SegmentSpeeds'] = None

In [ ]:
from tqdm import tqdm
for index, row in tqdm(df.iterrows(), total=df.shape[0]):
    df.at[index, 'SegmentSpeeds'] = get_segment_speeds(row['Line'], row['Line_time'])

df.head(5)

In [17]:
df['SegmentPoints'] = df['SegmentSpeeds'].apply(
    lambda segs: [tuple(pt.coords[0]) for pt, speed in segs] if isinstance(segs, list) else None
)
df['SegmentSpeeds_mps'] = df['SegmentSpeeds'].apply(
    lambda segs: [speed for pt, speed in segs] if isinstance(segs, list) else None
)

In [18]:
df = df.drop(columns=['SegmentSpeeds'])

In [ ]:
from pyproj import Transformer

transformer = Transformer.from_crs(3857, 4326, always_xy=True)

def transform_points(points):
    return [transformer.transform(*pt) for pt in points]

df['SegmentPoints'] = df['SegmentPoints'].apply(transform_points)
df.head(3)

In [ ]:
df.to_parquet('df.parquet', index=False)